In [ ]:
# pip install --upgrade pip
# pip install selenium ## make sure also install dirver for your OS
# pip install requests
# pip install beautifulsoup4

### install ffmpeg

In [ ]:
import os, re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  

In [ ]:
# Define selenium Chrome Driver
chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--lang=zh")
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
def get_91_vidoe_lst_from_start_page(start_page):
    def get_href(video_div):
        return 'https://91porny.com/' + video_div.find_all("a")[1].get('href')

    driver.get(start_page)
    html = driver.page_source
    soup = BeautifulSoup(html)
    video_divs = soup.find_all("div", {"class": "colVideoList"})
    return [get_href(video_div) for video_div in video_divs]


In [ ]:
def get_clean_title(soup):
    title = soup.title.string
    title = title.replace('高清', '')
    title = title.replace('原创', '')
    title = title.replace('91视频', '')
    title = title.replace('91视频', '')
    title = title.replace('91自拍', '')
    title = title.replace('国产自拍', '')
    title = title.replace('[原创]', '')
    cleanString = re.sub('\W+','', title)
    return cleanString

def get_m3u8_of_page(soup):
    m3u8_url = [x.get("data-src")  for x in soup.findAll("video")][0]
    return m3u8_url

def extract91page(video_page_91):
    driver.get(video_page_91)
    html = driver.page_source
    soup = BeautifulSoup(html)
    title= get_clean_title(soup)
    m3u8_url = get_m3u8_of_page(soup)
    return title, m3u8_url

In [ ]:
## Sinagle video download
# title, m3u8_url = extract91page('https://91porny.com/video/view/f1d323f1b2d0b955e773') 
# os.system(f"ffmpeg -i '{m3u8_url}' -acodec copy -vcodec copy {title}.mp4")

In [ ]:
## get recent-favorite 91 video list
start_page='https://91porny.com/video/category/recent-favorite'
lst = get_91_vidoe_lst_from_start_page(start_page)
print(lst)

OUTPUT_FOLDER = "~/download_91_videos" # MAKE SURE creat this folder

for url in lst:
    try:
        title, m3u8_url = extract91page(url) 
        os.system(f"ffmpeg -i '{m3u8_url}' -acodec copy -vcodec copy {OUTPUT_FOLDER}/{title}.mp4")
    except:
        print("err"+url)